In [1]:
import sys
import time
import pickle
import pandas as pd
from pprint import pprint
from collections import Counter

In [2]:
def unpickle(filename):
    ''' open file '''
    with open(filename, 'r') as picklefile:
        old_data = pickle.load(picklefile)
    return old_data

In [3]:
basedir = "./pkls/"

In [81]:
#mbid_tracks = {}
#with open("mbid_tracks_1.csv", 'r') as mtf:
#    for line in mtf:
#        line = line.strip()
#        segs = line.split(",")
#        key = segs[0]
#        track = segs[1]
#        mbid_tracks[key] = track

In [82]:
#print len(mbid_tracks.keys())

83690


In [31]:
mbid_songs = {}
with open("mbid_songs_1.csv", 'r') as msf:
    for line in msf:
        line = line.strip()
        segs = line.split(",")
        key = segs[0]
        track = segs[1]
        mbid_songs[key] = track

In [32]:
print len(mbid_songs.keys())

9839


In [34]:
usersongs = []
linelimit = 8#11
currentline = 0
with open("mbids_1.txt", 'r') as mf:
    for line in mf:
        if currentline >= linelimit:
            break
        else:
            uss = []
            for mbid in line.split():
                song = mbid_songs[mbid]
                #print track
                if song != "":
                    #print track
                    uss.append(song)
            usersongs.append(uss)
        currentline += 1

In [35]:
with open("lastfm_usersongs.txt", 'w') as lusf:
    for uss in usersongs:
        #print "hello"
        for song in uss:
            lusf.write(song + " ")
        lusf.write("\n")

In [72]:
#usertracks = []
#with open("lastfm_usertracks.txt", 'r') as lutf:
#    for line in lutf:
#        line = line.strip()
#        uts = []
#        for l in line.split():
#            uts.append(l)
#        usertracks.append(uts)

In [11]:
#print usertracks[0]
#print usersongs[0]

In [12]:
kmeans = unpickle(basedir + "kmeans40.pkl")

In [13]:
with open(basedir + "zscore_df_csv.csv", 'r') as tf:
    tracks = pd.read_csv(tf, index_col=0)

###When matching on trackid

In [89]:
#userclusters = []
#t1 = time.time()
#for uts in usertracks:
#    ucs = []
#    for trackid in uts:
#        if trackid in tracks.track_id.values:
#            print trackid
#            track = tracks[tracks["track_id"] == trackid]
#            track = track.iloc[:,1:] #leave out track_id
#            cluster = kmeans.predict(track)
#            ucs.append(cluster[0])#only one track used in predict
#    userclusters.append(ucs) 
#print "predict time:", time.time()-t1

predict time: 32.6945610046


###For matching on songid

In [15]:
with open(basedir + "titles_csv3.csv", 'r') as tf:
    titles = pd.read_csv(tf, index_col=0)

In [18]:
titles.head()

,track_id,title,artist_id,artist,song_id
index,,,,,
0,TRBGPHG12903CE6CC3,The Law Gonna Step On You (1931),ARFXRHR1187B98FF09,Bo Carter,SORUUEV12A58A7B9FA
1,TRBGPYK128F42796E1,Des Vôtres,ARQDTOS12086C11443,Fredericks_ Goldman_ Jones,SOEZDOH12A8AE4787A
2,TRBGPJP128E078ED20,Crazy,AR12F2S1187FB56EEF,Aerosmith,SOOOWIC12A6701C7E5
3,TRBGPXH128F428C912,Chiove,AR7LIU31187B98EF11,Rita Chiarelli,SOLSWBA12A8C141B9A
4,TRBGPSV12903CA9C25,Rude Bwoy Love (feat. Dj Fly_ Dj Traxx_ T.Will...,ARPGCHN1187B9A2831,Nicky B_ Naëlle,SOHNJQL12AB018CC5C


In [16]:
with open("mismatch_tracks.txt", 'r') as mtf:
    bad_tracks = mtf.read().split()

print bad_tracks[:5]

btset = set(bad_tracks)

['TRAAAJO12903CAAC69', 'TRAAAPK128E0786D96', 'TRAAEJQ128F92C484E', 'TRAAGPJ128F428CD1B', 'TRAAHER128F92D0354']


In [24]:
#usersongs.append(['SOOOWIC12A6701C7E5'])

In [36]:
userclusters = []
t1 = time.time()
for uss in usersongs:
    ucs = []
    for song in uss:
        if song in titles.song_id.values:
            print song
            trackmatches = titles[titles["song_id"] == song]
            trackmatches = trackmatches["track_id"].values #could be more than one
            for tm in trackmatches:
                if tm not in btset:
                    track = tracks[tracks["track_id"] == tm]
                    track = track.iloc[:,1:] #leave out track_id
                    cluster = kmeans.predict(track)
                    ucs.append(cluster[0])#only one track used in predict
    userclusters.append(ucs) 
print "predict time:", time.time()-t1

SOMGPML12A8C13AE8C
SOEBCBI12AF72A154F
SOHOOXT12A67ADA073
SOAWCYO12A8151DF97
SOJHUQN12A6D4F8EDE
SODKJWI12A8151BD74
SOBRZCG12A6702187D
SOZOEYP12AB0188C9D
SOMGPML12A8C13AE8C
SOZTULX12A6701E7EF
SOLSJHM12A8C139B46
SOQIDEF12A8C139B23
SOZBKLM12A58A78692
SOREDLN12A58A7CD54
SOEIQUY12AF72A086A
predict time: 1.37127780914


In [37]:
clustercounts = []
clusterfreqs = []
for ucs in userclusters:
    num_clusters = kmeans.cluster_centers_.shape[0]
    indcounts = [0 for cl in range(num_clusters)]
    indfreqs = [0 for cl in range(num_clusters)]
    counts = Counter(ucs)
    countsum = float(sum(counts.values()))
    for name, c in counts.iteritems():
        indcounts[name] = c
        indfreqs[name] = c/countsum
    clustercounts.append(indcounts)
    clusterfreqs.append(indfreqs)
    if len(ucs) > 30:
        print k, len(ucs), counts
        print indcounts
        print indfreqs

In [38]:
with open(basedir+"lastfm_clustercounts.txt", 'w') as cc:
    for clist in clustercounts:
        for c in clist:
            cc.write(str(c) + " ")
        cc.write("\n")

with open(basedir+"lastfm_clusterfreqs.txt", 'w') as cf:
    for flist in clusterfreqs:
        for f in flist:
            cf.write(str(f) + " ")
        cf.write("\n")